In [139]:
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, AdaBoostClassifier,VotingClassifier,RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

In [142]:
import warnings
warnings.filterwarnings('ignore')

# data load and primary check on data

In [84]:
df = pd.read_csv('Company_Data.csv')
df.head()

<IPython.core.display.Javascript object>

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [86]:
df.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [87]:
df.shape

(400, 11)

# converting categorical data using label encoder

In [88]:
lab_enco = LabelEncoder()
df['ShelveLoc'] = lab_enco.fit_transform(df['ShelveLoc'])

In [89]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,Yes,Yes
1,11.22,111,48,16,260,83,1,65,10,Yes,Yes
2,10.06,113,35,10,269,80,2,59,12,Yes,Yes
3,7.40,117,100,4,466,97,2,55,14,Yes,Yes
4,4.15,141,64,3,340,128,0,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,Yes,Yes
396,6.14,139,23,3,37,120,2,55,11,No,Yes
397,7.41,162,26,12,368,159,2,40,18,Yes,Yes
398,5.94,100,79,7,284,95,0,50,12,Yes,Yes


In [90]:
df['Urban']=lab_enco.fit_transform(df['Urban'])
df['US']=lab_enco.fit_transform(df['US'])

In [91]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,1,65,10,1,1
2,10.06,113,35,10,269,80,2,59,12,1,1
3,7.40,117,100,4,466,97,2,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,1,1
396,6.14,139,23,3,37,120,2,55,11,0,1
397,7.41,162,26,12,368,159,2,40,18,1,1
398,5.94,100,79,7,284,95,0,50,12,1,1


# adding a categorical column for sales status

In [95]:
sales_status=[]
for i in df['Sales']:
    if i>=7.5:
        sales_status.append('High sales')
    else:
        sales_status.append('low sales')
        
df['sales_status'] = sales_status

In [96]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales_status
0,9.50,138,73,11,276,120,0,42,17,1,1,High sales
1,11.22,111,48,16,260,83,1,65,10,1,1,High sales
2,10.06,113,35,10,269,80,2,59,12,1,1,High sales
3,7.40,117,100,4,466,97,2,55,14,1,1,low sales
4,4.15,141,64,3,340,128,0,38,13,1,0,low sales
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,1,1,High sales
396,6.14,139,23,3,37,120,2,55,11,0,1,low sales
397,7.41,162,26,12,368,159,2,40,18,1,1,low sales
398,5.94,100,79,7,284,95,0,50,12,1,1,low sales


In [97]:
x = df.iloc[:,1:11]
y = df.iloc[:,-1]

In [98]:
x

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,138,73,11,276,120,0,42,17,1,1
1,111,48,16,260,83,1,65,10,1,1
2,113,35,10,269,80,2,59,12,1,1
3,117,100,4,466,97,2,55,14,1,1
4,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,1,33,14,1,1
396,139,23,3,37,120,2,55,11,0,1
397,162,26,12,368,159,2,40,18,1,1
398,100,79,7,284,95,0,50,12,1,1


In [99]:
y

0      High sales
1      High sales
2      High sales
3       low sales
4       low sales
          ...    
395    High sales
396     low sales
397     low sales
398     low sales
399    High sales
Name: sales_status, Length: 400, dtype: object

# applying RF for prediction 

In [113]:
kfold = KFold(n_splits=10)
model = RandomForestClassifier(n_estimators=100,max_depth=5,max_features=5,random_state=42)
result = cross_val_score(model,x,y,cv=kfold)
result.mean()                '''this hyper parameters are iterated and optimum values are kept in this step'''

0.8125

In [114]:
model.fit(x,y)

RandomForestClassifier(max_depth=5, max_features=5, random_state=42)

In [116]:
df['predicted_sales_status']=model.predict(x)

In [117]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales_status,predicted_sales_status
0,9.50,138,73,11,276,120,0,42,17,1,1,High sales,low sales
1,11.22,111,48,16,260,83,1,65,10,1,1,High sales,High sales
2,10.06,113,35,10,269,80,2,59,12,1,1,High sales,High sales
3,7.40,117,100,4,466,97,2,55,14,1,1,low sales,High sales
4,4.15,141,64,3,340,128,0,38,13,1,0,low sales,low sales
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,1,1,High sales,High sales
396,6.14,139,23,3,37,120,2,55,11,0,1,low sales,low sales
397,7.41,162,26,12,368,159,2,40,18,1,1,low sales,low sales
398,5.94,100,79,7,284,95,0,50,12,1,1,low sales,low sales


In [118]:
df['sales_status'].value_counts()

low sales     201
High sales    199
Name: sales_status, dtype: int64

In [119]:
df['predicted_sales_status'].value_counts()

High sales    205
low sales     195
Name: predicted_sales_status, dtype: int64

# applying ensemble technique for increasing the accuracy of model

# Bagging

In [126]:
dt = DecisionTreeClassifier()
model1 = BaggingClassifier(base_estimator=dt,n_estimators=100,max_features=5,random_state=42)
result1 = cross_val_score(model1,x,y,cv=kfold)
result1.mean()

0.8100000000000002

In [127]:
model1.fit(x,y)
df['predicted_bagging_SalesStatus']=model1.predict(x)
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales_status,predicted_sales_status,predicted_bagging_SalesStatus
0,9.50,138,73,11,276,120,0,42,17,1,1,High sales,low sales,High sales
1,11.22,111,48,16,260,83,1,65,10,1,1,High sales,High sales,High sales
2,10.06,113,35,10,269,80,2,59,12,1,1,High sales,High sales,High sales
3,7.40,117,100,4,466,97,2,55,14,1,1,low sales,High sales,low sales
4,4.15,141,64,3,340,128,0,38,13,1,0,low sales,low sales,low sales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,1,1,High sales,High sales,High sales
396,6.14,139,23,3,37,120,2,55,11,0,1,low sales,low sales,low sales
397,7.41,162,26,12,368,159,2,40,18,1,1,low sales,low sales,low sales
398,5.94,100,79,7,284,95,0,50,12,1,1,low sales,low sales,low sales


# Boosting

In [137]:
model2 = AdaBoostClassifier(n_estimators=200,random_state=42)
result2 = cross_val_score(model2,x,y,cv=kfold)
result2.mean()

0.8300000000000001

In [138]:
model2.fit(x,y)
df['predicted_boosting_SalesStatus']=model2.predict(x)
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales_status,predicted_sales_status,predicted_bagging_SalesStatus,predicted_boosting_SalesStatus
0,9.50,138,73,11,276,120,0,42,17,1,1,High sales,low sales,High sales,High sales
1,11.22,111,48,16,260,83,1,65,10,1,1,High sales,High sales,High sales,High sales
2,10.06,113,35,10,269,80,2,59,12,1,1,High sales,High sales,High sales,High sales
3,7.40,117,100,4,466,97,2,55,14,1,1,low sales,High sales,low sales,low sales
4,4.15,141,64,3,340,128,0,38,13,1,0,low sales,low sales,low sales,low sales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,1,1,High sales,High sales,High sales,High sales
396,6.14,139,23,3,37,120,2,55,11,0,1,low sales,low sales,low sales,low sales
397,7.41,162,26,12,368,159,2,40,18,1,1,low sales,low sales,low sales,low sales
398,5.94,100,79,7,284,95,0,50,12,1,1,low sales,low sales,low sales,low sales


# voting classifier

In [140]:
estimator=[]

model3 = DecisionTreeClassifier()
estimator.append(('DTC',model3))

model4 = LogisticRegression()
estimator.append(('LR',model4))

model5 = SVC()
estimator.append(('SVC',model5))

ensable = VotingClassifier(estimator)

In [143]:
result3 = cross_val_score(ensable,x,y,cv=kfold)
result3.mean()

0.7975000000000001

# Boosting yielded model with best accuracy